In [1]:
import os, sys, logging, importlib, math
import rasterio, affine

import geopandas as gpd
import pandas as pd

from zipfile import ZipFile
from affine import Affine
from rasterio import features
from rasterio.mask import mask
from rasterio.features import rasterize

from gep_results import gep_results

# Mapping electrification
We need to convert certain fields in the results to a raster; these fields need to be calculated from the existing data

1. Population already electrified 2030
    a. Rasterize field **ElecPop**
2. New population electrified by grid 2030
    a. Filter data where **FinalElecCode2030** == 1.0
    b. Set **NewConnections2025** = 0 where **FinalElecCode2025** == 99.0
    c. Combine **NewConnections2030** with **NewConnections2025**
3. New population electrified by off grid 2030
    a. Filter data where **FinalElecCode2030** != 1.0
    b. Set **NewConnections2025** = 0 where **FinalElecCode2025** != **FinalElecCode2030**
    c. Combine **NewConnections2030** with **NewConnections2025**

In [ ]:
xx_so = gep_results('et-1')
result = xx_so.join_results()

In [ ]:
for field in ['ElecPop','GridPop2030','OffGridPop2030']:
    res = 0.05
    outFile = os.path.join(xx_so.countryClustersFolder, "rasterized_gep_%s_%s.tif" % (field, res))
    xx_so.rasterize_results(result, outFile, field=field, res=res, dtype='float64')

In [ ]:
subset = result[['geometry','ElecPop','Pop','Pop2030','GridPop2030','OffGridPop2030','FinalElecCode2030']]
origShape = subset.shape[0]
origPop = subset['Pop2030'].sum()
for pop in [10, 100, 1000, 10000]:
    curSub = subset[subset['Pop2030'] > pop]
    print("%s - %s" % (curSub.shape[0]/origShape, curSub['Pop2030'].sum()/origPop))

In [ ]:
curSub = subset[subset['Pop2030'] > 100]
curSub.to_file('/media/gost/DATA1/GEP/Clusters/et-1/final_clusters_minPop_100.shp')

In [ ]:
[print(x) for x in result.columns]

In [ ]:
xx_et = gep_results('et-1')
#etRes = xx_et.join_results()
outFile = os.path.join(xx_et.countryClustersFolder, "rasterized_gep_sorted2.tif")
xx_et.rasterize_results(etRes, outFile, res=0.01)



In [12]:
xx_so = gep_results('ke-1')
res = xx_so.join_results()

for resolution in [0.01]:
    outFile = os.path.join(xx_so.countryClustersFolder, "rasterized_gep_%s.tif" % resolution)
    xx_so.rasterize_results(res, outFile, res=resolution)

/home/gost/anaconda3/envs/Geog/lib/python3.7/site-packages/ipykernel_launcher.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [29]:
# process all remaining countries
clustersFolder = '/media/gost/DATA/GEP/Clusters'
zipFiles = []
processedCountries = []
for root, folders, files in os.walk(clustersFolder):    
    processedCountries.append([f for f in folders])
    for f in files:
        if f[-4:] == ".zip":
            zipFiles.append(f[:-4])
            
processedCountries = [item for sublist in processedCountries for item in sublist]

In [30]:
for country in zipFiles:
    if not country in processedCountries:
        print(f"Processing {country}")
        try:
            xx_so = gep_results(country)
            res = xx_so.join_results()
            outFile = os.path.join(xx_so.countryClustersFolder, "rasterized_gep_%s.tif" % resolution)
            xx_so.rasterize_results(res, outFile, res=resolution)
        except:
            print(f'error processing {country}')

Processing id-1
error processing id-1
Processing kh-1


/home/gost/anaconda3/envs/Geog/lib/python3.7/site-packages/ipykernel_launcher.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Processing km-1


/home/gost/anaconda3/envs/Geog/lib/python3.7/site-packages/ipykernel_launcher.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Processing lr-1


...

/home/gost/anaconda3/envs/Geog/lib/python3.7/site-packages/ipykernel_launcher.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Processing gn-1


/home/gost/anaconda3/envs/Geog/lib/python3.7/site-packages/ipykernel_launcher.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Processing ne-1


/home/gost/anaconda3/envs/Geog/lib/python3.7/site-packages/ipykernel_launcher.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [32]:
# Convert 0 in data to no data
allImages = []
for root, folders, files in os.walk(clustersFolder):
    for f in files:
        if f[-6:] == "01.tif":
            allImages.append(os.path.join(root, f))

for inFile in allImages:
    outFile = inFile.replace(".tif", "_noData.tif")
    print(f'gdal_translate -of GTiff -a_nodata 0 {inFile} {outFile}')

gdal_translate -of GTiff -a_nodata 0 /media/gost/DATA/GEP/Clusters/ao-1/rasterized_gep_0.01.tif /media/gost/DATA/GEP/Clusters/ao-1/rasterized_gep_0.01_noData.tif
gdal_translate -of GTiff -a_nodata 0 /media/gost/DATA/GEP/Clusters/bd-1/rasterized_gep_0.01.tif /media/gost/DATA/GEP/Clusters/bd-1/rasterized_gep_0.01_noData.tif
gdal_translate -of GTiff -a_nodata 0 /media/gost/DATA/GEP/Clusters/bf-1/rasterized_gep_0.01.tif /media/gost/DATA/GEP/Clusters/bf-1/rasterized_gep_0.01_noData.tif
gdal_translate -of GTiff -a_nodata 0 /media/gost/DATA/GEP/Clusters/bi-1/rasterized_gep_0.01.tif /media/gost/DATA/GEP/Clusters/bi-1/rasterized_gep_0.01_noData.tif
gdal_translate -of GTiff -a_nodata 0 /media/gost/DATA/GEP/Clusters/bj-1/rasterized_gep_0.01.tif /media/gost/DATA/GEP/Clusters/bj-1/rasterized_gep_0.01_noData.tif
gdal_translate -of GTiff -a_nodata 0 /media/gost/DATA/GEP/Clusters/bw-1/rasterized_gep_0.01.tif /media/gost/DATA/GEP/Clusters/bw-1/rasterized_gep_0.01_noData.tif
gdal_translate -of GTiff -a_

In [ ]:
find /media/gost/DATA/GEP/Clusters -name "*0.01_noData.tif" > tifFiles.txt
gdalbuildvrt combined_gep_rasterized_01.vrt -input_file_list tifFiles.txt